# 반려견 안구질환 진단 예측 프로젝트 - 두부는 단단

In [ ]:
# 라이브러리 임포트
import os
import glob
import json
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
import os
import glob
import json
import pandas as pd

import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold

data = {
    "결막염": ["유", "무"],
    "궤양성각막질환": ["상", "하", "무"],
    "백내장": ["초기", "비성숙", "성숙", "무"],
    "비궤양성각막질환": ["상", "하", "무"],
    "색소침착성각막염": ["유", "무"],
    "안검내반증": ["유", "무"],
    "안검염": ["유", "무"],
    "안검종양": ["유", "무"],
    "유루증": ["유", "무"],
    "핵경화": ["유", "무"]
}
lesion_names = {
    '결막염': 'conjunctivitis',
    '궤양성각막질환': 'corneal_ulcer',
    '백내장': 'cataract',
    '비궤양성각막질환': 'non_ulcerative_keratitis',
    '색소침착성각막염': 'pigmentary_keratitis',
    '안검내반증': 'entropion',
    '안검염': 'blepharitis',
    '안검종양': 'eyelid_tumor',
    '유루증': 'uveitis',
    '핵경화': 'nuclear_sclerosis'
}

# dataframe
def create_dataframe_from_json(directory_path):
    data = []
    for file_path in glob.glob(os.path.join(directory_path, "*.json")):
        with open(file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
            # Extract required fields
            image_meta = json_data.get('images', {}).get('meta', {})
            label_info = json_data.get('label', {})
            data.append({
                # 데이터프레임 생성을 위한 정보 추출
                'breed': json_data["images"]["meta"]["breed"],
                'age': json_data["images"]["meta"]["age"],
                'gender' : json_data["images"]["meta"]["gender"],
                'eye_position' : json_data["images"]["meta"]["eye_position"],
                'lesions' : json_data["label"]["label_disease_nm"],
                'label_disease_lv_1' : json_data["label"]["label_disease_lv_1"],
                'label_disease_lv_2' : json_data["label"]["label_disease_lv_2"],
                'label_disease_lv_3' : json_data["label"]["label_disease_lv_3"],
                'img_path' : os.path.join(directory_path, json_data["label"]["label_filename"])
            })
    return pd.DataFrame(data)

# 증상 유무
def map_and_check_disease_presence(row):
    # Define a function to map values to 0 or 1
    def map_to_binary(value):
        return 0 if value == '무' else 1
    
    # Apply the function to the specified columns and check if any value is 1
    for column in ['label_disease_lv_1', 'label_disease_lv_2', 'label_disease_lv_3']:
        if map_to_binary(row[column]) == 1:
            return 1
    return 0

# 확장자 명 없는 파일경로 수정

def check_and_correct_image_paths(df, path=0):
    """
    Checks if the images paths exist in the 'img_path' column of the DataFrame.
    If an image path does not exist, it appends the corresponding extension (.png or .jpg) to the image path in the DataFrame.

    Parameters:
        df (DataFrame): The DataFrame containing the 'img_path' column.
        path (int): If set to 0, '.png' will be appended to the image path if it does not exist, 
                    otherwise '.jpg' will be appended. Default is 0.
    """
    for idx, img_path in enumerate(df['img_path']):
        if not os.path.exists(img_path):
            if path == 0:
                df.at[idx, 'img_path'] += '.png'
            else:
                df.at[idx, 'img_path'] += '.jpg'

# Example usage:
# check_and_correct_image_paths(df, path=0)



def prepare_image_data(df, target_size=(100, 100)):
    """
    Preprocesses image data from the DataFrame by loading images, resizing them, and converting them to numpy arrays.

    Parameters:
        df (DataFrame): The DataFrame containing the 'img_path' column.
        target_size (tuple): Target size for resizing images. Default is (100, 100).

    Returns:
        X_aug (ndarray): Numpy array containing preprocessed image data.
        Y_aug (ndarray): Numpy array containing labels.
        test_datagen (ImageDataGenerator): ImageDataGenerator object fitted to the preprocessed image data.
    """
    X_aug = []
    for img_path in df['img_path']:
        img = Image.open(img_path).resize(target_size)
        img_array = np.asarray(img)
        X_aug.append(img_array)
    X_aug = np.stack(X_aug, axis=0)
    Y_aug = np.array(df.iloc[:, -1:])
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen.fit(X_aug)
    
    return X_aug, Y_aug

# Example usage:
# X_aug, Y_aug, test_datagen = prepare_image_data(df)





# model
def create_model():
    model = Sequential()
    model.add(Conv2D(16, kernel_size = (3,3), input_shape = [100, 100, 3], activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2)))

    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))

    model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))
    
    model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))

    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(7, activation='sigmoid'))

    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

    model.compile(loss = 'sparse_categorical_crossentropy',
                 optimizer = optimizer,
                  metrics = ['accuracy'])
    print(model.summary())
    return model

def train_model(model, X_train, Y_train, EPOCHS):
    early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='auto')
    print("Training model...")
    history = model.fit(X_train, Y_train, validation_split=0.2, batch_size = 64, epochs = EPOCHS,
                        callbacks = [reduce_lr, early_stop])
    print("h2")
    return history

def plot_model_training_curve(history):
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Model Accuracy', 'Model Loss'])
    fig.add_trace(
        go.Scatter(
            y=history.history['accuracy'], name='train_acc'), row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_accuracy'], name='val_acc'), row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['loss'], name='train_loss'), row=1, col=2)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_loss'], name='val_loss'), row=1, col=2)
    fig.show()





# 주어진 데이터로 모든 폴더에서 데이터프레임 생성
all_dfs = []
for disease, symptoms in data.items():
    for symptom in symptoms:
        for dataset_type in ["1.Training", "2.Validation"]:
            directory_path = f"D:\\반려동물 안구질환 데이터\\{dataset_type}\\{disease}\\{symptom}"
            print(directory_path)
            df = create_dataframe_from_json(directory_path)
            all_dfs.append(df)
            
            # Apply the combined function to create 'path' column
            df['path'] = df.apply(map_and_check_disease_presence, axis=1)
            # 확장자 명 없는 파일경로 수정
            check_and_correct_image_paths(df, path=0)
            
            X_aug, Y_aug = prepare_image_data(df)
            
            epochs = 50
            model=create_model()
            print('------------------------------------------------------------------------')
            history = train_model(model, X_aug, Y_aug, EPOCHS=epochs)
            if dataset_type == "1.Training":
                plot_model_training_curve(history)
            else:
                model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
                loss, accuracy = model.evaluate(X_aug, Y_aug, verbose=0)
                print(f'Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
                model_file='안검종양모델'+'.h5'
                model.save(model_file)
                print(f'Test Loss: {loss}')
                print(f'Test Accuracy: {accuracy}')


# 생성된 데이터프레임들을 하나로 합치기
merged_df = pd.concat(all_dfs, ignore_index=True)

# 생성된 데이터프레임 출력
print(merged_df)


In [ ]:
import os
import glob
import json
import pandas as pd

import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from plotly.subplots import make_subplots


data = {
    "결막염": ["유", "무"],
    "궤양성각막질환": ["상", "하", "무"],
    "백내장": ["초기", "비성숙", "성숙", "무"],
    "비궤양성각막질환": ["상", "하", "무"],
    "색소침착성각막염": ["유", "무"],
    "안검내반증": ["유", "무"],
    "안검염": ["유", "무"],
    "안검종양": ["유", "무"],
    "유루증": ["유", "무"],
    "핵경화": ["유", "무"]
}
lesion_names = {
    '결막염': 'conjunctivitis',
    '궤양성각막질환': 'corneal_ulcer',
    '백내장': 'cataract',
    '비궤양성각막질환': 'non_ulcerative_keratitis',
    '색소침착성각막염': 'pigmentary_keratitis',
    '안검내반증': 'entropion',
    '안검염': 'blepharitis',
    '안검종양': 'eyelid_tumor',
    '유루증': 'uveitis',
    '핵경화': 'nuclear_sclerosis'
}

# dataframe
def create_dataframe_from_json(directory_path):
    data = []
    for file_path in glob.glob(os.path.join(directory_path, "*.json")):
        with open(file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
            # Extract required fields
            image_meta = json_data.get('images', {}).get('meta', {})
            label_info = json_data.get('label', {})
            data.append({
                # 데이터프레임 생성을 위한 정보 추출
                'breed': json_data["images"]["meta"]["breed"],
                'age': json_data["images"]["meta"]["age"],
                'gender' : json_data["images"]["meta"]["gender"],
                'eye_position' : json_data["images"]["meta"]["eye_position"],
                'lesions' : json_data["label"]["label_disease_nm"],
                'label_disease_lv_1' : json_data["label"]["label_disease_lv_1"],
                'label_disease_lv_2' : json_data["label"]["label_disease_lv_2"],
                'label_disease_lv_3' : json_data["label"]["label_disease_lv_3"],
                'img_path' : os.path.join(directory_path, json_data["label"]["label_filename"])
            })
    return pd.DataFrame(data)

# 증상 유무
def map_and_check_disease_presence(row):
    # Define a function to map values to 0 or 1
    def map_to_binary(value):
        return 0 if value == '무' else 1
    
    # Apply the function to the specified columns and check if any value is 1
    for column in ['label_disease_lv_1', 'label_disease_lv_2', 'label_disease_lv_3']:
        if map_to_binary(row[column]) == 1:
            return 1
    return 0

# 확장자 명 없는 파일경로 수정

def check_and_correct_image_paths(df, path=0):
    """
    Checks if the images paths exist in the 'img_path' column of the DataFrame.
    If an image path does not exist, it appends the corresponding extension (.png or .jpg) to the image path in the DataFrame.

    Parameters:
        df (DataFrame): The DataFrame containing the 'img_path' column.
        path (int): If set to 0, '.png' will be appended to the image path if it does not exist, 
                    otherwise '.jpg' will be appended. Default is 0.
    """
    for idx, img_path in enumerate(df['img_path']):
        if not os.path.exists(img_path):
            if path == 0:
                df.at[idx, 'img_path'] += '.png'
            else:
                df.at[idx, 'img_path'] += '.jpg'

# Example usage:
# check_and_correct_image_paths(df, path=0)



def prepare_image_data(df, target_size=(100, 100)):
    """
    Preprocesses image data from the DataFrame by loading images, resizing them, and converting them to numpy arrays.

    Parameters:
        df (DataFrame): The DataFrame containing the 'img_path' column.
        target_size (tuple): Target size for resizing images. Default is (100, 100).

    Returns:
        X_aug (ndarray): Numpy array containing preprocessed image data.
        Y_aug (ndarray): Numpy array containing labels.
        test_datagen (ImageDataGenerator): ImageDataGenerator object fitted to the preprocessed image data.
    """
    X_aug = []
    for img_path in df['img_path']:
        img = Image.open(img_path).resize(target_size) # default 100, 100
        img_array = np.asarray(img)
        X_aug.append(img_array)
    X_aug = np.stack(X_aug, axis=0)
    Y_aug = np.array(df.iloc[:, -1:])
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen.fit(X_aug)
    
    return X_aug, Y_aug

# Example usage:
# X_aug, Y_aug, test_datagen = prepare_image_data(df)





# model
def create_model():
    model = Sequential()
    model.add(Conv2D(16, kernel_size = (3,3), input_shape = [100, 100, 3], activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2)))

    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))

    model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))
    
    model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))

    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

    model.compile(loss = 'binary_crossentropy',
                 optimizer = optimizer,
                  metrics = ['binary_accuracy'])
    print(model.summary())
    return model

def train_model(model, X_train, Y_train, EPOCHS):
    early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='auto')
    print("Training model...")
    history = model.fit(X_train, Y_train, validation_split=0.2, batch_size = 64, epochs = EPOCHS,
                        callbacks = [reduce_lr, early_stop])
    print("h2")
    return history





# 주어진 데이터로 모든 폴더에서 데이터프레임 생성

for disease, symptoms in data.items():
    for dataset_type in ["1.Training", "2.Validation"]:
        all_dfs = []
        for symptom in symptoms:
            directory_path = f"D:\\반려동물 안구질환 데이터\\{dataset_type}\\{disease}\\{symptom}"
            print(directory_path)
            df = create_dataframe_from_json(directory_path)
            all_dfs.append(df)
            
        # Apply the combined function to create 'path' column
        df['path'] = df.apply(map_and_check_disease_presence, axis=1)
        # 확장자 명 없는 파일경로 수정
        check_and_correct_image_paths(df, path=0)
        
        X_aug, Y_aug = prepare_image_data(df)
        
        
        if dataset_type == "1.Training":
            epochs = 50
            model=create_model()
            history = train_model(model, X_aug, Y_aug, EPOCHS=epochs)
            model_file=f'{lesion_names[disease]}'+'.h5'
            model.save(model_file)
        elif dataset_type == "2.Validation":
            loss, accuracy = model.evaluate(X_aug, Y_aug, verbose=0)
            print(disease + ' Validation')
            print(f'Test Loss: {loss}')
            print(f'Test Accuracy: {accuracy}')


# # 생성된 데이터프레임들을 하나로 합치기
# merged_df = pd.concat(all_dfs, ignore_index=True)
# 
# # 생성된 데이터프레임 출력
# print(merged_df)


In [ ]:
dfs = [pd.DataFrame,'bbb']
a, b =dfs
a

# GoogleNet

In [ ]:
import os
import glob
import json
import pandas as pd

import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold
from plotly.subplots import make_subplots

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D


data = {
    "결막염": ["유", "무"],
    "궤양성각막질환": ["상", "하", "무"],
    "백내장": ["초기", "비성숙", "성숙", "무"],
    "비궤양성각막질환": ["상", "하", "무"],
    "색소침착성각막염": ["유", "무"],
    "안검내반증": ["유", "무"],
    "안검염": ["유", "무"],
    "안검종양": ["유", "무"],
    "유루증": ["유", "무"],
    "핵경화": ["유", "무"]
}
lesion_names = {
    '결막염': 'conjunctivitis',
    '궤양성각막질환': 'corneal_ulcer',
    '백내장': 'cataract',
    '비궤양성각막질환': 'non_ulcerative_keratitis',
    '색소침착성각막염': 'pigmentary_keratitis',
    '안검내반증': 'entropion',
    '안검염': 'blepharitis',
    '안검종양': 'eyelid_tumor',
    '유루증': 'uveitis',
    '핵경화': 'nuclear_sclerosis'
}

def create_dataframes(disease):
    all_dfs = [] # all_dfs[0]: train_df, all_dfs[1]: valid_df
    for dataset_type in ["1.Training", "2.Validation"]:
        df_sets = []
        for symptom in data[disease]:
            directory_path = f"D:\\반려동물 안구질환 데이터\\{dataset_type}\\{disease}\\{symptom}"
            print(directory_path)
            df = create_dataframe_from_json(directory_path)
            df_sets.append(df)
        concatenated_df = pd.concat(df_sets)
        concatenated_df.reset_index(drop=True, inplace=True)
        all_dfs.append(concatenated_df)
            
    # disease lv로 증상 유무 판단 -> 누락 이미지 경로 채우기
    for df in all_dfs:       
        df['path'] = df.apply(map_and_check_disease_presence, axis=1)
        # 확장자 명 없는 파일경로 수정
        check_and_correct_image_paths(df, path=0)
        
    train_df, valid_df = all_dfs
    return train_df, valid_df

# dataframe
def create_dataframe_from_json(directory_path):
    data = []
    for file_path in glob.glob(os.path.join(directory_path, "*.json")):
        with open(file_path, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
            # Extract required fields
            image_meta = json_data.get('images', {}).get('meta', {})
            label_info = json_data.get('label', {})
            data.append({
                # 데이터프레임 생성을 위한 정보 추출
                'breed': json_data["images"]["meta"]["breed"],
                'age': json_data["images"]["meta"]["age"],
                'gender' : json_data["images"]["meta"]["gender"],
                'eye_position' : json_data["images"]["meta"]["eye_position"],
                'lesions' : json_data["label"]["label_disease_nm"],
                'label_disease_lv_1' : json_data["label"]["label_disease_lv_1"],
                'label_disease_lv_2' : json_data["label"]["label_disease_lv_2"],
                'label_disease_lv_3' : json_data["label"]["label_disease_lv_3"],
                'img_path' : os.path.join(directory_path, json_data["label"]["label_filename"])
            })
    return pd.DataFrame(data)

# 증상 유무
def map_and_check_disease_presence(row):
    # Define a function to map values to 0 or 1
    def map_to_binary(value):
        return 0 if value == '무' else 1
    
    # Apply the function to the specified columns and check if any value is 1
    for column in ['label_disease_lv_1', 'label_disease_lv_2', 'label_disease_lv_3']:
        if map_to_binary(row[column]) == 1:
            return '1'
    return '0'

# 확장자 명 없는 파일경로 수정

def check_and_correct_image_paths(df, path=0):
    """
    Checks if the images paths exist in the 'img_path' column of the DataFrame.
    If an image path does not exist, it appends the corresponding extension (.png or .jpg) to the image path in the DataFrame.

    Parameters:
        df (DataFrame): The DataFrame containing the 'img_path' column.
        path (int): If set to 0, '.png' will be appended to the image path if it does not exist, 
                    otherwise '.jpg' will be appended. Default is 0.
    """
    for idx, img_path in enumerate(df['img_path']):
        if not os.path.exists(img_path):
            if path == 0:
                df.at[idx, 'img_path'] += '.png'
            else:
                df.at[idx, 'img_path'] += '.jpg'
    

# Example usage:
# check_and_correct_image_paths(df, path=0)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_data_generators(train_df, valid_df):
    """
    Create data generators for training and validation dataframes.

    Parameters:
        train_df (DataFrame): DataFrame containing training data.
        valid_df (DataFrame): DataFrame containing validation data.

    Returns:
        train_generator (DirectoryIterator): Data generator for training data.
        val_generator (DirectoryIterator): Data generator for validation data.
    """
    # 데이터 전처리
    datagen = ImageDataGenerator(rescale=1./255)

    train_generator = datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col='img_path',
        y_col='path',
        target_size=(100, 100),  # InceptionV3 모델의 입력 크기
        batch_size=32,
        class_mode='binary'  # 이진 클래스 분류
    )

    val_generator = datagen.flow_from_dataframe(
        dataframe=valid_df,
        x_col='img_path',
        y_col='path',
        target_size=(100, 100),
        batch_size=32,
        class_mode='binary'
    )

    return train_generator, val_generator

# Usage example:
# train_generator, val_generator = create_data_generators(train_df, valid_df)

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

def conduct_googlenet():
    # 구글넷 모델 불러오기
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(100, 100, 3))
    
    # 모델 구성
    model = Sequential()
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # 클래스 수에 맞게 출력층 설정
    
    # 기존의 컴파일 및 학습 코드 추가
    
    # 모델 컴파일
    model.compile(optimizer=Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model
        

def train_model_with_callbacks(model, train_generator, val_generator):
    """
    Train the model with callbacks for model checkpointing and early stopping.

    Parameters:
        model (tf.keras.Model): The model to train.
        train_generator (DirectoryIterator): Data generator for training data.
        val_generator (DirectoryIterator): Data generator for validation data.
        epochs (int): Number of epochs for training. Default is 30.
        checkpoint_path (str): Path to save the best model checkpoint. Default is "best_model.keras".
        patience (int): Number of epochs with no improvement after which training will be stopped. Default is 5.

    Returns:
        history (tf.keras.callbacks.History): Object containing training history.
    """
    
    
    # ModelCheckpoint callback: Save the best model checkpoint based on validation loss
    checkpoint = ModelCheckpoint(f'{lesion_names[disease]}.keras', monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    # EarlyStopping callback: Stop training if validation loss does not improve for 'patience' epochs
    early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)

    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=30,
        validation_data=val_generator,
        validation_steps=len(val_generator),
        callbacks=[checkpoint, early_stop]  # Add callbacks
    )

    return history

# Usage example:
# history = train_model_with_callbacks(model, train_generator, val_generator)




# 주어진 데이터로 모든 폴더에서 데이터프레임 생성

for disease in data.keys():
    print(disease)
    train_df, valid_df = create_dataframes(disease)
    print(disease + ' model')
    train_generator, val_generator = create_data_generators(train_df, valid_df)
    model = conduct_googlenet()
    history = train_model_with_callbacks(model, train_generator, val_generator)


결막염
D:\반려동물 안구질환 데이터\1.Training\결막염\유
D:\반려동물 안구질환 데이터\1.Training\결막염\무
D:\반려동물 안구질환 데이터\2.Validation\결막염\유
D:\반려동물 안구질환 데이터\2.Validation\결막염\무
결막염 model
Found 19198 validated image filenames belonging to 2 classes.
Found 2403 validated image filenames belonging to 2 classes.
Epoch 1/30


C:\aepython\eye_lesions\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step - accuracy: 0.9503 - loss: 0.1431
Epoch 1: val_loss improved from inf to 2.89248, saving model to conjunctivitis.keras
600/600 ━━━━━━━━━━━━━━━━━━━━ 226s 343ms/step - accuracy: 0.9503 - loss: 0.1430 - val_accuracy: 0.9309 - val_loss: 2.8925
Epoch 2/30

Epoch 2: val_loss improved from 2.89248 to 0.00000, saving model to conjunctivitis.keras


C:\Users\asiae\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 3/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step - accuracy: 0.9891 - loss: 0.0375
Epoch 3: val_loss did not improve from 0.00000
600/600 ━━━━━━━━━━━━━━━━━━━━ 201s 334ms/step - accuracy: 0.9891 - loss: 0.0375 - val_accuracy: 0.8906 - val_loss: 0.5368
Epoch 4/30

Epoch 4: val_loss did not improve from 0.00000
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 75us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0000e+00
Epoch 5/30
 88/600 ━━━━━━━━━━━━━━━━━━━━ 2:49 331ms/step - accuracy: 0.9939 - loss: 0.0356

In [ ]:
train_df['path_str'] = train_df['path'].astype(str)
valid_df['path_str'] = valid_df['path'].astype(str)

# 데이터 전처리
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='img_path',
    y_col='path_str',
    target_size=(100, 100),  # InceptionV3 모델의 입력 크기
    batch_size=32,
    class_mode='binary'  # 이진 클래스 분류
)

val_generator = datagen.flow_from_dataframe(
    dataframe=valid_df,
    x_col='img_path',
    y_col='path_str',
    target_size=(100, 100),
    batch_size=32,
    class_mode='binary'
)


In [ ]:
train_generator[0][1]

In [ ]:

# 구글넷 모델 불러오기
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import GlobalAveragePooling2D


# 구글넷 모델 불러오기
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# 모델 구성
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # 클래스 수에 맞게 출력층 설정

# 기존의 컴파일 및 학습 코드 추가

# 모델 컴파일
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])



model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# ModelCheckpoint 콜백: 검증 손실이 낮아질 때 모델 저장
checkpoint = ModelCheckpoint("best_model.keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# EarlyStopping 콜백: 검증 손실이 5 에포크 동안 개선되지 않으면 학습 종료
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_df) // 32,
    epochs=30,
    validation_data=val_generator,
    validation_steps=len(valid_df) // 32,
    callbacks=[checkpoint, early_stop]  # 콜백 함수 추가
)


In [ ]:
model.save('GoogleNet_eye.h5')

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_image(model_path, image_path, labels):
    # 모델 불러오기
    model = tf.keras.models.load_model(model_path)

    # 이미지 불러오기 및 전처리
    img = image.load_img(image_path, target_size=(100, 100))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.

    # 예측 수행
    predictions = model.predict(img_array)

    # 예측 결과에서 가장 높은 확률을 가진 클래스 인덱스 찾기
    predicted_class_index = np.argmax(predictions)
    # 해당 클래스의 라벨
    predicted_label = labels[predicted_class_index]
    # 해당 클래스의 확률
    predicted_prob = predictions[0][predicted_class_index]

    return predicted_label, predicted_prob

# 예측된 클래스의 라벨과 해당 라벨에 대한 확률을 출력하는 함수
def print_prediction_result(predicted_label, predicted_prob):
    print("Predicted label:", predicted_label)
    print("Predicted probability:", predicted_prob)

# 예시 이미지 경로
image_paths = ['C:\aepython\eye_lesions\models\결막염 테스트\dog.jpg','C:\aepython\eye_lesions\models\결막염 테스트\dog2.jpg',
               'C:\aepython\eye_lesions\models\결막염 테스트\dog3.jpg','C:\aepython\eye_lesions\models\결막염 테스트\dog5.jpg','C:\aepython\eye_lesions\models\결막염 테스트\dog6.jpg','C:\aepython\eye_lesions\models\결막염 테스트\dog7.jpg']

# 모델 경로
model_path = 'GoogleNet.h5'

# 라벨 설정 (이진 분류일 경우)
labels = ['0', '1']

# 이미지 예측 수행
for image_path in image_paths:
    predicted_label, predicted_prob = predict_image(model_path, image_path, labels)
    # 결과 출력
    print_prediction_result(predicted_label, predicted_prob)


In [ ]:
model.summary()


# resnet

In [ ]:
from keras.applications.resnet50 import preprocess_input

def create_generators(preprocess_func=None):
    #rescale changes pixels from 1-255 integers to 0-1 floats suitable for neural nets
    rescale = 1./255
    if preprocess_func is not None:
        #https://stackoverflow.com/questions/48677128/what-is-the-right-way-to-preprocess-images-in-keras-while-fine-tuning-pre-traine
        #no need to rescale if using Keras in-built ResNet50 preprocess_func: https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py#L157
        rescale = None

    train_datagen=ImageDataGenerator(
        rescale = rescale
    )
    #Keras has this two-part process of defining generators. 
    #First the generic properties above, then the actual generators with filenames and all.
    train_generator=train_datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col="img_path", #the name of column containing image filename in dataframe
        y_col="path", #the y-col in dataframe
        batch_size=batch_size, 
        # shuffle=False,
        class_mode="binary", #categorical if multiple. then y_col can be list or tuple also 
        #classes=lbls, #list of ouput classes. if not provided, inferred from data
        target_size=(100,100),
        subset='training') #the subset of data from the ImageDataGenerator definition above. The validation_split seems to produce these 2 values.

    valid_generator=train_datagen.flow_from_dataframe(
        dataframe=valid_df,
        x_col="img_path",
        y_col="path",
        batch_size=batch_size,
        # shuffle=False,
        class_mode="binary",
        #classes=lbls,
        target_size=(100,100), #gave strange error about tuple cannot be interpreted as integer
        subset='validation') #the subset of data from the ImageDataGenerator definition above. The validation_split seems to produce these 2 values.

    return train_generator, valid_generator, train_datagen



train_generator, valid_generator, train_datagen = create_generators()

class_map = {v: k for k, v in train_generator.class_indices.items()}???????

In [ ]:
from keras.applications import resnet50

train_generator, valid_generator, train_datagen = create_generators(preprocess_func = resnet50.preprocess_input)

In [ ]:
#the total number of images we have:
train_size = len(train_generator.filenames)
#train_steps is how many steps per epoch Keras runs the genrator. One step is batch_size*images
train_steps = train_size/batch_size
#use 2* number of images to get more augmentations in. some do, some dont. up to you
train_steps = int(2*train_steps)
#same for the validation set
valid_size = len(valid_generator.filenames)
valid_steps = valid_size/batch_size
valid_steps = int(2*valid_steps) 